# **RNN**

---



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create a working folder and cd to it.
!mkdir -p /content/drive/MyDrive/Learning/EIE4122/Lab2
%cd /content/drive/MyDrive/Learning/EIE4122/Lab2

/content/drive/MyDrive/Learning/EIE4122/Lab2


In [ ]:
# Require gdown to download files from Google Drive
!pip install gdown

In [ ]:
!echo "Start decompression. It will take few minutes"
!tar zxf data.tgz --exclude="._*"

Start decompression. It will take few minutes


In [ ]:
# Make sure that decompression is successful. The command "du" lists the size of folders.
!du -h .data/imdb/aclImdb

2.0M	.data/imdb/aclImdb/test/neg
2.2M	.data/imdb/aclImdb/test/pos
25M	.data/imdb/aclImdb/test
2.3M	.data/imdb/aclImdb/train/pos
2.1M	.data/imdb/aclImdb/train/neg
68M	.data/imdb/aclImdb/train
94M	.data/imdb/aclImdb


In [ ]:
# Install torch 2.0.1 and torchtext 0.5.0
!pip install torch==2.0.1 torchtext==0.5.0
# you can safely ignore the message "ERROR: pip's dependency resolver..." for this lab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
# Define input TEXT and output LABEL
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = 'spacy',tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
# Because data has been uploaded to ".data/imdb/aclImdb/", this step will only take 1 minute.
# This step should be quick as PyTorch only needs to know the structure of the data in the
# folder ".data/imdb/aclImdb". It will not actually load the data, which will be done by
# the Dataloader object (see code below)
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, path='.data/imdb/aclImdb/')

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 2780
Number of testing examples: 2780


In [ ]:
print(vars(train_data.examples[0]))

{'text': ['"', 'Cinema', 'is', 'the', 'ultimate', 'pervert', 'art', '.', 'It', 'does', "n't", 'give', 'you', 'what', 'you', 'desire', ';', 'it', 'tells', 'you', 'how', 'to', 'desire', '.', '"<br', '/><br', '/>So', 'begins', '"', 'The', 'Pervert', "'s", 'Guide', 'to', 'Cinema', ',', '"', 'in', 'which', 'Slovenian', 'philosopher', 'and', 'psychoanalyst', 'Slavoj', 'Zizek', 'applies', 'his', 'Freudian', '/', 'Lacanian', 'brain', '-', 'scalpel', 'to', 'world', 'cinema', '.', 'This', 'film', 'in', 'three', 'parts', 'is', 'the', 'second', 'feature', 'documentary', 'directed', 'by', 'Sophie', 'Fiennes', '(', 'yes', ',', 'sister', 'of', 'Ralph', 'and', 'Joseph', ')', ',', 'and', 'it', 'is', 'a', 'notable', 'accomplishment', ',', 'clocking', 'in', 'at', '2', '1/2', 'hours', 'of', 'talk', 'from', 'one', 'man', 'and', 'yet', 'remaining', 'humorous', 'and', 'engaging', 'throughout', '.', 'In', 'essence', ',', 'it', 'is', 'an', 'extended', 'film', 'lecture', ',', 'and', 'one', 'of', 'the', 'best', 

In [ ]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 1946
Number of validation examples: 834
Number of testing examples: 2780


In [ ]:
# MAX_VOCAB_SIZE = 25_000    # In Python 25_000 means 25000
MAX_VOCAB_SIZE = 50_000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 31932
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 22385), (',', 21678), ('.', 18541), ('a', 12623), ('and', 12392), ('of', 11250), ('to', 10503), ('is', 8776), ('in', 7070), ('it', 5694), ('I', 5689), ('that', 5523), ('"', 5220), ("'s", 4948), ('this', 4672), ('-', 4498), ('/><br', 3904), ('was', 3736), ('as', 3541), ('with', 3319)]


In [ ]:
print(TEXT.vocab.itos[:10])      # Print from the beginning (0th) to the 9th position

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)
print(LABEL.vocab.stoi['neg'])

defaultdict(None, {'neg': 0, 'pos': 1})
0


In [ ]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):

        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        #text = [sent len, batch size]

        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]

        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        return self.fc(hidden.squeeze(0))  # Remove the first dim in hidden to return [batch_size, hid_dim]

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [ ]:
# Count the number of trainable parameters in our model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,285,105 trainable parameters


In [ ]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, max_samples=2500):

    epoch_loss = 0
    epoch_acc = 0

    model.train()
    #for batch in iterator:
    for i, batch in enumerate(iterator):
        if i > max_samples:                               # Use less samples to reduce training time
            break
        optimizer.zero_grad()                             # Initialize gradient to 0 for current batch
        predictions = model(batch.text).squeeze(1)        # Call RNN.forward() and get return object
        loss = criterion(predictions, batch.label)        # Compute binary cross-entropy
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()                                   # Compute gradient
        optimizer.step()                                  # Update the weights by gradient descent
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, max_samples=2500):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():                                    # No gradient will be needed
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)       # Call RNN.forward()
            loss = criterion(predictions, batch.label)       # Compute binary cross entropy loss
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 12s
	Train Loss: 0.694 | Train Acc: 50.10%
	 Val. Loss: 0.696 |  Val. Acc: 49.00%
Epoch: 02 | Epoch Time: 2m 11s
	Train Loss: 0.693 | Train Acc: 50.48%
	 Val. Loss: 0.696 |  Val. Acc: 48.88%
Epoch: 03 | Epoch Time: 2m 15s
	Train Loss: 0.694 | Train Acc: 50.05%
	 Val. Loss: 0.696 |  Val. Acc: 48.88%
Epoch: 04 | Epoch Time: 2m 10s
	Train Loss: 0.693 | Train Acc: 50.50%
	 Val. Loss: 0.696 |  Val. Acc: 48.88%
Epoch: 05 | Epoch Time: 2m 4s
	Train Loss: 0.693 | Train Acc: 50.18%
	 Val. Loss: 0.696 |  Val. Acc: 48.77%
Epoch: 06 | Epoch Time: 2m 5s
	Train Loss: 0.693 | Train Acc: 49.85%
	 Val. Loss: 0.696 |  Val. Acc: 48.88%
Epoch: 07 | Epoch Time: 2m 3s
	Train Loss: 0.694 | Train Acc: 50.35%
	 Val. Loss: 0.696 |  Val. Acc: 48.88%
Epoch: 08 | Epoch Time: 2m 9s
	Train Loss: 0.694 | Train Acc: 50.06%
	 Val. Loss: 0.696 |  Val. Acc: 48.77%
Epoch: 09 | Epoch Time: 2m 4s
	Train Loss: 0.693 | Train Acc: 50.25%
	 Val. Loss: 0.696 |  Val. Acc: 48.66%
Epoch: 10 | Epoch Time: 

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.693 | Test Acc: 51.84%


# **Bidirectional Two-layer LSTM**

---



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create a working folder and cd to it.
!mkdir -p /content/drive/MyDrive/Learning/EIE4122/Lab2
%cd /content/drive/MyDrive/Learning/EIE4122/Lab2

/content/drive/MyDrive/Learning/EIE4122/Lab2


In [ ]:
# Require gdown to download files from Google Drive
!pip install gdown

In [ ]:
# Download the reduced-size IMDB dataset
!if [ ! -f data.tgz ]; then gdown https://drive.google.com/uc?id=1EL6YlIEs1IbUDjrZvrZ_9-C0C4gYoryZ; fi

In [ ]:
!echo "Start decompression. It will take few minutes"
!tar xf data.tgz --exclude="._*"

Start decompression. It will take few minutes


In [ ]:
# Install torch 2.0.1 and torchtext 0.5.0
!pip install torch==2.0.1 torchtext==0.5.0
# you can safely ignore the message "ERROR: pip's dependency resolver..." for this lab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 112.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
# This step should be quick as PyTorch only needs to know the structure of the data in the
# folder ".data/imdb/aclImdb". It will not actually load the data, which will be done by
# the Dataloader object (see code below)
print('Loading data from .data/imdb/aclImdb')
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, path='.data/imdb/aclImdb/')
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Loading data from .data/imdb/aclImdb
Number of training examples: 2780
Number of testing examples: 2780


In [ ]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        #text = [sent len, batch size]

        embedded = self.dropout(self.embedding(text))
        #embedded = [sent len, batch size, emb dim]

        #pack sequence, lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors

        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]

        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)
print(model.embedding.weight.data.size())

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7289, -0.7336,  1.5624,  ..., -0.5592, -0.4480, -0.6476],
        ...,
        [ 0.0914,  1.5196,  0.4670,  ...,  0.6393, -0.0332,  0.0185],
        [-0.6290,  0.4650, -0.7165,  ..., -1.3171,  2.0381, -2.0497],
        [-1.1222, -0.0240, -1.0878,  ..., -0.4948, -0.3874,  0.0339]])
torch.Size([25002, 100])


In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0
    model.train()

    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss: 0.465 | Train Acc: 78.61%
	 Val. Loss: 0.635 |  Val. Acc: 67.30%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss: 0.391 | Train Acc: 82.69%
	 Val. Loss: 0.713 |  Val. Acc: 69.20%
Epoch: 03 | Epoch Time: 0m 4s
	Train Loss: 0.406 | Train Acc: 81.43%
	 Val. Loss: 0.709 |  Val. Acc: 66.07%
Epoch: 04 | Epoch Time: 0m 4s
	Train Loss: 0.349 | Train Acc: 84.08%
	 Val. Loss: 0.654 |  Val. Acc: 71.65%
Epoch: 05 | Epoch Time: 0m 4s
	Train Loss: 0.307 | Train Acc: 86.90%
	 Val. Loss: 0.651 |  Val. Acc: 70.87%
Epoch: 06 | Epoch Time: 0m 4s
	Train Loss: 0.249 | Train Acc: 89.80%
	 Val. Loss: 0.685 |  Val. Acc: 72.10%
Epoch: 07 | Epoch Time: 0m 4s
	Train Loss: 0.252 | Train Acc: 89.47%
	 Val. Loss: 0.935 |  Val. Acc: 72.54%
Epoch: 08 | Epoch Time: 0m 4s
	Train Loss: 0.209 | Train Acc: 91.58%
	 Val. Loss: 1.091 |  Val. Acc: 73.33%
Epoch: 09 | Epoch Time: 0m 4s
	Train Loss: 0.219 | Train Acc: 90.40%
	 Val. Loss: 0.863 |  Val. Acc: 71.54%
Epoch: 10 | Epoch Time: 0m 4

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.679 | Test Acc: 64.45%


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "The movie is awesome!.")

0.7324489951133728

In [ ]:
predict_sentiment(model, "This film is terrible!.")

0.34864652156829834